## Class data

In [3]:
import random

class Sentiment:
    NEGATIVE='NEGATIVE'
    POSITIVE='POSITIVE'
    NEUTRAL='NEUTRAL'

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
    
    def get_sentiment(self):
        if(self.score<=2):
            return Sentiment.NEGATIVE
        elif(self.score==3):
            return Sentiment.NEUTRAL
        else: #score>=4
            return Sentiment.POSITIVE

class ReviewContainer:
    def __init__(self,reviews):
        self.reviews=reviews
    
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evenly_distribute(self):
        negative=list(filter(lambda x:x.sentiment==Sentiment.NEGATIVE,self.reviews))
        positive=list(filter(lambda x:x.sentiment==Sentiment.POSITIVE,self.reviews))
        positive_shrunk=positive[:len(negative)]
        self.reviews=negative+positive_shrunk
        random.shuffle(self.reviews)

## Load data

In [10]:
import json

filename='Books_small_10000.json'

reviews=[]
with open(filename) as f:
    for line in f:
        review=json.loads(line)
        reviews.append(Review(review['reviewText'],review['overall']))

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

## Prep data

In [4]:
from sklearn.model_selection import train_test_split
training,test=train_test_split(reviews,test_size=0.33,random_state=42)
train_cont=ReviewContainer(training)
train_cont.evenly_distribute()
test_cont=ReviewContainer(test)
test_cont.evenly_distribute()



In [5]:

train_x=train_cont.get_text()
train_y=train_cont.get_sentiment()

test_x=test_cont.get_text()
test_y=test_cont.get_sentiment()

## Bags of words vectorization

In [6]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
#vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer()
train_x_vectors=vectorizer.fit_transform(train_x)
test_x_vectors=vectorizer.transform(test_x)

### SVM

In [7]:
from sklearn import svm
clf_svm=svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors,train_y)
print(test_x[0])
clf_svm.predict(test_x_vectors[0])


This was only my second book by R.L. Mathewson, but what I have discovered is that the author has quite the sense of humour!I loved the constant bickering between our H/h, Connor and Rory, and the fact that, when I wasn't chuckling with something I had read, I found myself with a constant smile on my face.Connor and Rory first meet when they are just children, and spend their early years and into adulthood being a thorn in the others side. Their being forced to work together will bring new confrontations, but will also bring to the fore emotions that have been bubbling under the surface all these years.If you like your romance with a touch of madcap comedy, then this is the book for you! There are moments that will make you laugh, moments that will make you smile, moments that might even bring a tear to your eye, and even a few moments that will have you fanning yourself!This is a feel good read, and the perfect book for a summer afternoon lazing about.


array(['NEGATIVE'], dtype='<U8')

### Decision Tree

In [8]:
from sklearn.tree import DecisionTreeClassifier
clf_dec=DecisionTreeClassifier()
clf_dec.fit(train_x_vectors,train_y)
clf_dec.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

### Naive Bayes

In [9]:
from sklearn.naive_bayes import GaussianNB
clf_gnb=GaussianNB()
clf_gnb.fit(train_x_vectors.todense(),train_y)
clf_gnb.predict(test_x_vectors[0].todense())

array(['NEGATIVE'], dtype='<U8')

### Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression().fit(train_x_vectors,train_y)
clf_lr.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

### Evaluation

In [11]:
print(clf_svm.score(test_x_vectors,test_y))
print(clf_dec.score(test_x_vectors,test_y))
print(clf_gnb.score(test_x_vectors.todense(),test_y))
print(clf_lr.score(test_x_vectors,test_y))

0.8076923076923077
0.6322115384615384
0.6610576923076923
0.8052884615384616


In [18]:
#f1 scores
from sklearn.metrics import f1_score
f1_score(test_y,clf_svm.predict(test_x_vectors),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE])

array([0.80582524, 0.80952381])

### Test suite

In [24]:
test_set=["this book is good","boring book","bad book"]
new_set=vectorizer.transform(test_set)
clf_svm.predict(new_set)

array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')